In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
import os
import boto3
import time

In [4]:
df = pd.read_csv("s3a://data-workspace-will-prod/flavia-costa/base_modelos2/base_modelos.csv/*", header=None)

In [5]:
len(df)

22175696

In [6]:
df.columns = ['id_pessoa', 'id_modelo', 'value', 'description']

In [7]:
df['value'] = df['value'].fillna('')

In [8]:
df['dt_model_update'] = datetime.today().strftime('%Y-%m-%d')

In [9]:
print(df['id_modelo'].unique())

['ds_classif_behavior' 'ds_seg_consumo' 'ds_perfil_research' 'ds_persona']


In [10]:
df['ttld'] = np.where(df['description'] == 'Behavior', 30,
             np.where(df['description'] == 'Segm.MCC', 90,
             np.where(df['description'] == 'Perfis', 30,         
             np.where(df['description'] == 'Persona', 1000, 60 ))))

In [11]:
df.head()

,id_pessoa,id_modelo,value,description,dt_model_update,ttld
0,40caa0ecde15593dde26cec518de77602d9e2ae7182f6b...,ds_classif_behavior,4,Behavior,2023-08-17,30
1,40caa0ecde15593dde26cec518de77602d9e2ae7182f6b...,ds_seg_consumo,SUPERMERCADO - 6,Segm.MCC,2023-08-17,90
2,40caa0ecde15593dde26cec518de77602d9e2ae7182f6b...,ds_perfil_research,Multiplicador,Perfis,2023-08-17,30
3,40caa0ecde15593dde26cec518de77602d9e2ae7182f6b...,ds_persona,"5 - Mulher, Casada, 25 anos ou mais, cidade Gr...",Persona,2023-08-17,1000
4,a61b721f5327d3562987531e8db4770472b158712007dd...,ds_classif_behavior,,Behavior,2023-08-17,30


In [12]:
s3 = boto3.client('s3')
import random

In [17]:
k = 60000

lista_sample = random.choices(df['id_pessoa'].unique(), k=k)

df_sample = df[df['id_pessoa'].isin(lista_sample)]

print(len(df_sample))

start_time = time.perf_counter()

for i in df_sample["id_pessoa"].unique():
    
    path = 'flavia-costa/teste_json_v5/experience_' + i + '.json'
    df_json = json.dumps(df_sample[df_sample['id_pessoa'] == i].groupby("id_pessoa").apply(lambda x: x.drop(columns="id_pessoa").to_dict(orient="records")).to_dict(), indent=4)

    s3.put_object(Body=df_json, Bucket='data-workspace-will-prod', Key=path)
    
end_time = time.perf_counter()

elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
print("Customers: ", k)

252288
Elapsed time:  4975.101873991
Customers:  60000


In [18]:
print("Elapsed time: ", elapsed_time)

Elapsed time:  4975.101873991
